# Solving a simple nonlinear poroelastic problem

*[Written by: M. Pierre, 2024]* \
\
Let us come back to our first example and complicate it a little bit: we will now consider nonlinear elasticity $(K = K(\varepsilon))$ and take into account the compressibility of water $(\rho_f = \rho_f(p))$. \
The states equations read:
$$
\begin{array}{}
\sigma = K(\varepsilon)\varepsilon - bp, \\
\phi-\phi_0 = b\varepsilon - \frac{p}{N}.
\end{array}
$$
Suppose we impose a stress $\tilde{\sigma}$ from an initially unstressed state and keep undrained conditions ($m_f = m_{f,0}$):
$$
\begin{array}{}
\tilde{\sigma} = K(\varepsilon)\varepsilon - bp, \\
0 = m_f-m_{f,0} = \rho_f(p)\phi - \rho_f(0)\phi_0.
\end{array}
$$
We can take a power law for $K$ (let us say for instance $K(\varepsilon) = K_0\sqrt{\varepsilon}$), and take into account the compressibility of water through $\rho_f = \rho_{f,0}(1+p/K_f)$
$$
\begin{array}{}
\tilde{\sigma} = K\varepsilon^{3/2} - bp, \\
0 = \left(1+\frac{p}{K_f}\right)\left(\phi_0+b\varepsilon - \frac{p}{N}\right) - \phi_0.
\end{array}
$$
